In [1]:


import pandas as pd
import os, random, sys, copy
import torch, torch.nn as nn, numpy as np
from tqdm.notebook import tqdm
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
from nltk.tokenize import word_tokenize
import matplotlib.pyplot as plt

import csv
import os

import re



C:\Users\verda\AppData\Roaming\Python\Python311\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [3]:
wd = "NLP_Spring24"
os.chdir(wd)

FileNotFoundError: [WinError 2] The system cannot find the file specified: 'NLP_Spring24'

In [4]:
os.getcwd()

'C:\\Users\\verda\\Downloads\\NLP_Spring24'

In [5]:

# Directory containing CSV files
directory = 'spotify_songs.csv'
df = pd.read_csv(directory)
df_small = pd.DataFrame({
    'Artist': df['track_artist'],
    'Lyrics': df['lyrics']
})

df_small

,Artist,Lyrics
0,Barbie's Cradle,Minsan pa Nang ako'y napalingon Hindi ko alam ...
1,Steady Rollin,"The trees, are singing in the wind The sky blu..."
2,Bell Biv DeVoe,"NA Yeah, Spyderman and Freeze in full effect U..."
3,CeeLo Green,I really can't stay Baby it's cold outside I'v...
4,KARD,Get up out of my business You don't keep me fr...
...,...,...
18449,Qulinez,"Caught up in such a head rush, wide-eyed latel..."
18450,Nicki Minaj,"My anaconda don't, my anaconda don't My anacon..."
18451,Ponderosa Twins Plus One,"Bound, bound Bound, bound Bound to fall in lov..."
18452,Father MC,(Would you do for me) Sweetheart (Would you do...


In [6]:
df

,track_id,track_name,track_artist,lyrics,track_popularity,track_album_id,track_album_name,track_album_release_date,playlist_name,playlist_id,...,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,language
0,0017A6SJgTbfQVU2EtsPNo,Pangarap,Barbie's Cradle,Minsan pa Nang ako'y napalingon Hindi ko alam ...,41,1srJQ0njEQgd8w4XSqI4JQ,Trip,2001-01-01,Pinoy Classic Rock,37i9dQZF1DWYDQ8wBxd7xt,...,-10.068,1,0.0236,0.27900,0.011700,0.0887,0.566,97.091,235440,tl
1,004s3t0ONYlzxII9PLgU6z,I Feel Alive,Steady Rollin,"The trees, are singing in the wind The sky blu...",28,3z04Lb9Dsilqw68SHt6jLB,Love & Loss,2017-11-21,Hard Rock Workout,3YouF0u7waJnolytf9JCXf,...,-4.739,1,0.0442,0.01170,0.009940,0.3470,0.404,135.225,373512,en
2,00chLpzhgVjxs1zKC9UScL,Poison,Bell Biv DeVoe,"NA Yeah, Spyderman and Freeze in full effect U...",0,6oZ6brjB8x3GoeSYdwJdPc,Gold,2005-01-01,"Back in the day - R&B, New Jack Swing, Swingbe...",3a9y4eeCJRmG9p4YKfqYIx,...,-7.504,0,0.2160,0.00432,0.007230,0.4890,0.650,111.904,262467,en
3,00cqd6ZsSkLZqGMlQCR0Zo,Baby It's Cold Outside (feat. Christina Aguilera),CeeLo Green,I really can't stay Baby it's cold outside I'v...,41,3ssspRe42CXkhPxdc12xcp,CeeLo's Magic Moment,2012-10-29,Christmas Soul,6FZYc2BvF7tColxO8PBShV,...,-5.819,0,0.0341,0.68900,0.000000,0.0664,0.405,118.593,243067,en
4,00emjlCv9azBN0fzuuyLqy,Dumb Litty,KARD,Get up out of my business You don't keep me fr...,65,7h5X3xhh3peIK9Y0qI5hbK,KARD 2nd Digital Single ‘Dumb Litty’,2019-09-22,K-Party Dance Mix,37i9dQZF1DX4RDXswvP6Mj,...,-1.993,1,0.0409,0.03700,0.000000,0.1380,0.240,130.018,193160,en
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18449,7zXzWCVuz7whIuYZyEAlxt,Rising Like The Sun - Radio Mix,Qulinez,"Caught up in such a head rush, wide-eyed latel...",0,1l4aoukbPgi5u2OaE2R4Zj,Rising Like The Sun,2014-03-24,♥ EDM LOVE 2020,6jI1gFr6ANFtT8MmTvA2Ux,...,-5.778,0,0.0878,0.00555,0.000000,0.3350,0.211,128.012,208656,en
18450,7zycSpvjDcqh6YT1FEl2kY,Anaconda,Nicki Minaj,"My anaconda don't, my anaconda don't My anacon...",49,5qs8T6ZHSrnllnOuUk6muC,The Pinkprint (Deluxe Edition),2014-12-15,10er Playlist,1kEczIkZH8IgaWT2BiApxZ,...,-6.224,1,0.1800,0.06730,0.000006,0.2140,0.647,129.990,260240,en
18451,7zye9v6B785eFWEFYs13C2,Bound,Ponderosa Twins Plus One,"Bound, bound Bound, bound Bound to fall in lov...",40,1xdgLmTFMSyJyI5DJOOX7T,2+2+1 = (Digitally Remastered),2013-07-09,Sexy Soul 2020,5EMARioe9z9eKOeWIAC2JW,...,-6.457,0,0.0270,0.71500,0.000428,0.1150,0.657,142.218,191205,en
18452,7zyLObYw4QUKQDyZOb4J0Y,I'll Do 4 U (Re-Recorded / Remastered),Father MC,(Would you do for me) Sweetheart (Would you do...,36,14HYMxFhpgDIr9cci1u0kt,I'll Do 4 U (Re-Recorded / Remastered),2010-10-01,New Jack Swing/ R&B Hits: 1987 - 2002,4sji14lrB5bgcr51lPALYH,...,-4.920,0,0.0633,0.14300,0.000000,0.0720,0.810,109.536,223890,en


In [7]:
def clean_lyrics(lyrics):
    lyrics = str(lyrics)
    
    lyrics = lyrics.replace('\'', '')
            
    lyrics = re.sub(r'[\*\[|\(|\{].*\n*.*[\]\)\}\*]' , ' ', lyrics)
   
    for verse in ['verse', 'VERSE', 'Verse']:
        lyrics = re.sub(verse+' \d*', '', lyrics)
    
    for word in ['OUTRO', 'INSTRUMENTAL', 'PRE', 'HOOK',
                 'PRODUCED', 'REFRAIN', 'POST', 'REPEAT', '2x', '3x', '4x',
                 'CHORUS', 'INTRO', 'INTERLUDE']:
        lyrics = lyrics.replace(word, '')
        
    lyrics = re.sub(r'\n*Chorus:*.*' , ' ', lyrics)
    lyrics = re.sub(r'^Chorus:*.*' , ' ', lyrics)
    lyrics = re.sub(r'\nRepeat [C|c]horus:*.*' , ' ', lyrics)
    lyrics = re.sub(r'Intro[\s|\n|:].*', ' ', lyrics)
    lyrics = re.sub(r'-+.*[i|I]nstrumental.*-+', ' ', lyrics)
    lyrics = re.sub(r'\nBrief instrumental.*\n', ' ', lyrics)
    lyrics = re.sub(r'\nInstrumental', ' ', lyrics)
    lyrics = re.sub(r'\nInstrumental break', ' ', lyrics)
    lyrics = re.sub(r'\nInstrumental--', ' ', lyrics)
    lyrics = re.sub(r'\n~Instrumental~', ' ', lyrics)
    lyrics = re.sub(r'\n\[*Bridge:\[*', ' ', lyrics)
    lyrics = re.sub(r'Hook:.*', ' ', lyrics)
    lyrics = re.sub(r'Repeat\s.*', ' ', lyrics)
    lyrics = re.sub(r'\nRepeat$', ' ', lyrics)
    lyrics = re.sub(r'.*[P|p]roduced [B|b]y.*', ' ', lyrics)
    lyrics = re.sub(r'.*[W|w]ritten [B|b]y.*', ' ', lyrics)
    lyrics = re.sub(r'- musical interlude -', ' ', lyrics)
    lyrics = re.sub(r'\nRefrain:', ' ', lyrics)
    lyrics = re.sub(r'[^\w\s]', ' ', lyrics)
    lyrics = re.sub(r'\s+', ' ', lyrics)
    lyrics = lyrics.strip()
    lyrics = lyrics.lower()
    lyrics = re.sub(r'\s{2,}', ' ', lyrics)

    return lyrics

In [8]:
# df_clean = df_Small.apply(lambda row: clean_lyrics(row["Lyrics"]))  

df_small['Cleaned Lyrics'] = df_small["Lyrics"].apply(clean_lyrics) 

In [ ]:
print("hi")

In [9]:
!pip install transformers 


Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip available: 22.3.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [11]:
from transformers import pipeline 

In [12]:
emotion = pipeline('text-classification', model='SamLowe/roberta-base-go_emotions')

config.json:   0%|          | 0.00/1.92k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/380 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

In [120]:
emotion_labels = emotion("I hope we do good on this presentation")
emotion_labels


[{'label': 'optimism', 'score': 0.9282177090644836}]

In [106]:
df1 = pd.read_csv(directory)
df1.colums = ['track_artist','lyrics']
df1.head(10)

C:\Users\verda\AppData\Local\Temp\ipykernel_38900\4142612938.py:2: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  df1.colums = ['track_artist','lyrics']


,track_id,track_name,track_artist,lyrics,track_popularity,track_album_id,track_album_name,track_album_release_date,playlist_name,playlist_id,...,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,language
0,0017A6SJgTbfQVU2EtsPNo,Pangarap,Barbie's Cradle,Minsan pa Nang ako'y napalingon Hindi ko alam ...,41,1srJQ0njEQgd8w4XSqI4JQ,Trip,2001-01-01,Pinoy Classic Rock,37i9dQZF1DWYDQ8wBxd7xt,...,-10.068,1,0.0236,0.27900,0.011700,0.0887,0.566,97.091,235440,tl
1,004s3t0ONYlzxII9PLgU6z,I Feel Alive,Steady Rollin,"The trees, are singing in the wind The sky blu...",28,3z04Lb9Dsilqw68SHt6jLB,Love & Loss,2017-11-21,Hard Rock Workout,3YouF0u7waJnolytf9JCXf,...,-4.739,1,0.0442,0.01170,0.009940,0.3470,0.404,135.225,373512,en
2,00chLpzhgVjxs1zKC9UScL,Poison,Bell Biv DeVoe,"NA Yeah, Spyderman and Freeze in full effect U...",0,6oZ6brjB8x3GoeSYdwJdPc,Gold,2005-01-01,"Back in the day - R&B, New Jack Swing, Swingbe...",3a9y4eeCJRmG9p4YKfqYIx,...,-7.504,0,0.2160,0.00432,0.007230,0.4890,0.650,111.904,262467,en
3,00cqd6ZsSkLZqGMlQCR0Zo,Baby It's Cold Outside (feat. Christina Aguilera),CeeLo Green,I really can't stay Baby it's cold outside I'v...,41,3ssspRe42CXkhPxdc12xcp,CeeLo's Magic Moment,2012-10-29,Christmas Soul,6FZYc2BvF7tColxO8PBShV,...,-5.819,0,0.0341,0.68900,0.000000,0.0664,0.405,118.593,243067,en
4,00emjlCv9azBN0fzuuyLqy,Dumb Litty,KARD,Get up out of my business You don't keep me fr...,65,7h5X3xhh3peIK9Y0qI5hbK,KARD 2nd Digital Single ‘Dumb Litty’,2019-09-22,K-Party Dance Mix,37i9dQZF1DX4RDXswvP6Mj,...,-1.993,1,0.0409,0.03700,0.000000,0.1380,0.240,130.018,193160,en
5,00f9VGHfQhAHMCQ2bSjg3D,Soldier,James TW,"Hold your breath, don't look down, keep trying...",70,3GNzXsFbzdwM0WKCZtgeNP,Chapters,2019-04-26,urban contemporary,4WiB26kw0INKwbzfb5M6Tv,...,-6.157,1,0.0550,0.28000,0.000000,0.0975,0.305,147.764,224720,en
6,00FROhC5g4iJdax5US8jRr,Satisfy You,Diddy,All I want is somebody who's gonna love me for...,52,2dHr0LpUe6CNV5lNsr8x0W,Forever,1999-08-24,"Swingbeat (old skool), New Jack Swing, R&B, Hi...",3krpccUV68nBGAQbvHEZDC,...,-10.050,1,0.1850,0.59100,0.000000,0.1450,0.695,87.261,286441,en
7,00GfGwzlSB8DoA0cDP2Eit,Tender Lover,Babyface,Feels good Everybody Tender lover Tender love ...,36,51fAXJ5bMn7DRSunXQ6PMb,Tender Lover,1989-07-07,New Jack Swing,3ykXidKLz1eYPvuGoFlD1e,...,-6.346,1,0.0445,0.22600,0.000422,0.0513,0.687,102.459,259267,en
8,00Gu3RMpDW2vO9PjlMVFDL,Hide Away (feat. Envy Monroe),Blasterjaxx,"Don't run away, it's getting colder Our hearts...",42,5pqG85igfoeWcCDIsSi9x7,Hide Away (feat. Envy Monroe),2019-06-21,Big Room EDM - by Spinnin' Records,7xWdFCrU5Gka6qp1ODrSdK,...,-4.894,1,0.0421,0.02490,0.000000,0.3610,0.134,130.001,188000,en
9,00GxbkrW4m1Tac5xySEJ4M,Ti volevo dedicare (feat. J-AX & Boomdabash),Rocco Hunt,Ho una cosa da dirti da tempo Ma non ho mai t...,78,57L1NgMlfxscOxHhmfLjqg,Libertà,2019-08-30,Musica Italiana 2020 - Playlist Pop & Hip-Hop ...,6kVFIQBhLT4003iw2WWEv1,...,-6.058,1,0.0661,0.01040,0.000000,0.1920,0.271,120.002,208133,it


In [32]:
df.shape

(18454, 25)

In [33]:
simpleText = df[:200]
simpleText.shape

(200, 25)

In [56]:
def get_emotion(text):
    return(emotion(text))

In [121]:
import csv

def extract_first_514_words_from_csv(csv_file):
    """
    Extracts the first 514 words from the "lyrics" column of the first 10 songs in a CSV file.

    Args:
    csv_file (str): The path to the CSV file.

    Returns:
    list: A list containing the first 514 words from each song.
    """
    first_514_words_list = []

    # Open the CSV file
    with open(csv_file, 'r', newline='', encoding='utf-8') as csvfile:
        reader = csv.DictReader(csvfile)
        
        # Initialize counter
        song_count = 0
        
        # Iterate through each row in the CSV file
        for row in reader:
            if song_count >= 10:
                break  # Exit loop if 10 songs have been processed
            
            # Extract lyrics from the row
            lyrics = row['lyrics'].strip()
            
            # Split the lyrics into words
            words = lyrics.split()
            
            # Extract the first 514 words and join them back into a string
            first_514_words = ' '.join(words[:50])
            
            # Append the result to the list
            first_514_words_list.append(first_514_words)
            
            # Increment song count
            song_count += 1

    return first_514_words_list

# Usage example
csv_file = directory
first_514_words_list_for10 = extract_first_514_words_from_csv(csv_file)


for i in range(len(first_514_words_list_for10)):
    print(i+1,emotion(first_514_words_list_for10[i]))
   
    # print(len(first_514_words_list_for10[i]))
print("1","Song name - Pangarap")
print("2","Song name - I Feel Alive")
print("3","Song name - Poison")
print("4","Song name - Baby its cold")
print("5","Song name - Dumb Litty")
print("6","Song name - Soldier")
print("7","Song name - Satisfy You")
print("8","Song name - Tender Lover")
print("9","Song name - Hide Away (feat. Envy Monroe)")
print("10","Song name - Ti volevo dedicare (feat. J-AX & Boomdabash)")
   

1 [{'label': 'neutral', 'score': 0.9228407740592957}]
2 [{'label': 'love', 'score': 0.300457239151001}]
3 [{'label': 'neutral', 'score': 0.6847439408302307}]
4 [{'label': 'desire', 'score': 0.2048022747039795}]
5 [{'label': 'anger', 'score': 0.497537761926651}]
6 [{'label': 'fear', 'score': 0.844062089920044}]
7 [{'label': 'love', 'score': 0.6178298592567444}]
8 [{'label': 'love', 'score': 0.7586883902549744}]
9 [{'label': 'neutral', 'score': 0.6310687065124512}]
10 [{'label': 'neutral', 'score': 0.8603540062904358}]
1 Song name - Pangarap
2 Song name - I Feel Alive
3 Song name - Poison
4 Song name - Baby its cold
5 Song name - Dumb Litty
6 Song name - Soldier
7 Song name - Satisfy You
8 Song name - Tender Lover
9 Song name - Hide Away (feat. Envy Monroe)
10 Song name - Ti volevo dedicare (feat. J-AX & Boomdabash)
